In [1]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Algorithmes.music import music_method
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch


nbSources = 2 # Nombre de sources
nbSensors = 10 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = 10
T = 100
correlation_List = [0.4] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

phi_max = 90  # Définition de phi_max
rho = 1  # Résolution de la grille
G = np.arange(-phi_max, phi_max + rho, rho)  # Création de la grille

def angles_to_binary_vector(thetaList, G):
    """
    Convertit une liste d'angles en un vecteur binaire représentant la présence/absence d'angles dans la grille G.
    
    Args:
    - thetaList: Liste des angles d'arrivée pour un échantillon.
    - G: Grille des angles possibles.
    
    Returns:
    - Vecteur binaire des labels Z.
    """
    Z = np.zeros(len(G), dtype=int)
    for theta in thetaList:
        index = np.searchsorted(G, theta)  # Trouver l'indice de l'angle dans la grille
        if 0 <= index < len(G):  # S'assurer que l'indice est dans la plage valide
            Z[index] = 1
    return Z

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

# Paramètres pour la simulation
num_samples = 16290  # Nombre d'échantillons à générer
L = 10  # Nombre de capteurs
T = 100  # Nombre de points temporels
phi_max = 90  # φmax
rho = 1  # Résolution

# Grille des angles
G = np.arange(-phi_max, phi_max + rho, rho)

# Initialisation des tableaux pour les données
all_R_hat_with_phase = np.zeros((num_samples, 3, L, L))
all_Z = np.zeros((num_samples, len(G)))

for i in range(num_samples):
    # Générer les angles d'arrivée et la matrice X comme décrit précédemment
    thetaList = [np.random.uniform(-90, 90) for _ in range(2)]  # Exemple de génération d'angles
    varList = [np.random.uniform(0, 1000000000000) for _ in range(nbSources)]
    X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, signal_noise_ratio)  # Votre fonction pour générer X, ajoutez les paramètres nécessaires
    
    # Calculer R_hat étendue avec phase
    R_hat_with_phase = generate_R_hat_with_phase(X)
    all_R_hat_with_phase[i] = R_hat_with_phase
    
    # Calculer les labels Z pour les angles générés
    all_Z[i] = angles_to_binary_vector(thetaList, G)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, Z_train, Z_test = train_test_split(all_R_hat_with_phase, all_Z, test_size=0.2, random_state=42)


[[4.84086225e+11 2.16929407e+11]
 [2.16929407e+11 6.07566965e+11]]
[[6.27248249e+11 2.92827748e+11]
 [2.92827748e+11 8.54407429e+11]]
[[1.12248116e+11 2.05965075e+10]
 [2.05965075e+10 2.36204477e+10]]
[[1.26841684e+11 1.36376372e+11]
 [1.36376372e+11 9.16423638e+11]]
[[9.70173935e+11 3.41776673e+11]
 [3.41776673e+11 7.52515156e+11]]
[[4.56563802e+11 7.54788660e+10]
 [7.54788660e+10 7.79882678e+10]]
[[3.25962875e+11 1.59529888e+11]
 [1.59529888e+11 4.87973231e+11]]
[[8.10352638e+11 2.05453303e+11]
 [2.05453303e+11 3.25560885e+11]]
[[5.43913041e+11 2.68446703e+11]
 [2.68446703e+11 8.28069321e+11]]
[[9.22045294e+11 1.77842477e+11]
 [1.77842477e+11 2.14387153e+11]]
[[8.96993422e+11 2.65253986e+11]
 [2.65253986e+11 4.90246606e+11]]
[[4.95828499e+11 1.13253563e+11]
 [1.13253563e+11 1.61678504e+11]]
[[9.05397699e+11 2.47356998e+11]
 [2.47356998e+11 4.22366081e+11]]
[[7.45278662e+11 2.50808600e+11]
 [2.50808600e+11 5.27528804e+11]]
[[2.46074476e+11 2.07530699e+10]
 [2.07530699e+10 1.09390133e+

In [6]:
X_train = torch.from_numpy(X_train).to(dtype=torch.float32)
X_test = torch.from_numpy(X_test).to(dtype=torch.float32)
Z_train = torch.from_numpy(Z_train).to(dtype=torch.float32)
Z_test = torch.from_numpy(Z_test).to(dtype=torch.float32)

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class DOADataset(Dataset):
    def __init__(self, covariance_matrices, doa_labels):
        """
        Initialisation du Dataset DOA.

        Args:
        - covariance_matrices (np.ndarray): Matrices de covariance étendues avec la phase.
          Forme attendue: (nb_samples, nbSensors, nbSensors, 3) pour les parties réelle, imaginaire, et la phase.
        - doa_labels (np.ndarray): Labels DOA sous forme de vecteurs binaires.
          Forme attendue: (nb_samples, nb_labels), où nb_labels est le nombre de points dans la grille DOA.
        """
        self.covariance_matrices = covariance_matrices
        self.doa_labels = doa_labels

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
      cov_matrix = self.covariance_matrices[idx]
      angle = self.doa_labels[idx]

      # Vérification si cov_matrix est déjà un ndarray, sinon le convertir
      if not isinstance(cov_matrix, np.ndarray):
          cov_matrix = cov_matrix.numpy()  # Convertir en tableau NumPy si ce n'est pas déjà fait
    
      cov_matrix = torch.from_numpy(cov_matrix).float()
      angle = torch.tensor(angle).float()

      return cov_matrix, angle


# Exemple d'utilisation (remplacez R_hat_train/test et angles_train/test par vos données réelles)
train_dataset = DOADataset(X_train, Z_train)
test_dataset = DOADataset(X_test, Z_test)

# Création des DataLoaders pour l'entraînement et le test
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)



In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

# Note: we don't recommend using the "mps" device for this notebook.

print(f"Using {device} device")

Using cpu device


In [25]:
import torch
import torch.nn as nn

class DOACNN(nn.Module):
    def __init__(self, num_channels=3, num_classes=181):
        super(DOACNN, self).__init__()

        # Couches convolutionnelles 2D
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=3, stride=2)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1)
        self.conv10 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1)

        # Normalisation de taille 256
        self.norm2 = nn.BatchNorm2d(256)
        self.norm5 = nn.BatchNorm2d(256)
        self.norm8 = nn.BatchNorm2d(256)
        self.norm11 = nn.BatchNorm2d(256)

        self.relu3 =  nn.ReLU()
        self.relu6 =  nn.ReLU()
        self.relu9 =  nn.ReLU()
        self.relu12 =  nn.ReLU()
        self.relu15 =  nn.ReLU()
        self.relu18 =  nn.ReLU()
        self.relu21 =  nn.ReLU()

        # Couches Dropout
        self.dropout16 = nn.Dropout(0.2)
        self.dropout19 = nn.Dropout(0.2)
        self.dropout22 = nn.Dropout(0.2)

        # Couche d'aplatissement
        self.flatten13 = nn.Flatten()

        # Couches entièrement connectées (FC)
        self.fc14 = nn.Linear(in_features=256 , out_features=4096)#anciennement in_features=256*7*7 a voir avec nathan prq
        self.fc17 = nn.Linear(in_features=4096, out_features=2048)
        self.fc20 = nn.Linear(in_features=2048, out_features=1024)
        self.fc23 = nn.Linear(in_features=1024, out_features=num_classes)

        # Couche de sortie Sigmoid
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm2(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.norm5(x)
        x = self.relu6(x)
        x = self.conv7(x)
        x = self.norm8(x)
        x = self.relu9(x)
        x = self.conv10(x)
        x = self.norm11(x)
        x = self.relu12(x)
        x = self.flatten13(x)
        x = self.fc14(x)
        x = self.relu15(x)
        x = self.dropout16(x)
        x = self.fc17(x)
        x = self.relu18(x)
        x = self.dropout19(x)
        x = self.fc20(x)
        x = self.relu21(x)
        x = self.dropout22(x)
        x = self.fc23(x)
        x = self.sigmoid(x)

        return x

# Création d'une instance du modèle
model = DOACNN()
print(model)

DOACNN(
  (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(2, 2))
  (conv4): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv10): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (norm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (relu6): ReLU()
  (relu9): ReLU()
  (relu12): ReLU()
  (relu15): ReLU()
  (relu18): ReLU()
  (relu21): ReLU()
  (dropout16): Dropout(p=0.2, inplace=False)
  (dropout19): Dropout(p=0.2, inplace=False)
  (dropout22): Dropout(p=0.2, inplace=False)
  (flatten13): Flatten(start_dim=1, end_dim=-1)
  (fc14): Linear(in_features=256, out_features=4096, bias=T

In [26]:
import torch.optim as optim

learning_rate = 0.001
num_epochs = 10

# Définir la fonction de perte et l'optimiseur
criterion = nn.BCELoss()  # Utilisation de la perte d'entropie croisée pour une classification multi-classe
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()  # Mettre le modèle en mode d'entraînement

    running_loss = 0.0
    for inputs, labels in train_dataloader:  # train_dataloader est votre DataLoader d'entraînement
        optimizer.zero_grad()  # Réinitialiser les gradients

        # Forward pass
        outputs = model(inputs)

        # Calcul de la perte
        loss = criterion(outputs, labels)

        # Backpropagation et mise à jour des poids
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Afficher la perte moyenne pour chaque époque
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_dataloader)}')

print('Entraînement terminé!')

torch.Size([64, 3, 10, 10])


C:\Users\legra\AppData\Local\Temp\ipykernel_22708\1484115762.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  angle = torch.tensor(angle).float()


torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 10, 10])
torch.Size([64, 3, 1

In [ ]:
Z_train.size()

torch.Size([13032, 181])